In [ ]:
!pip install pennylane

In [1]:
import pennylane as qml
from pennylane import numpy as np

# Introduction to Applications of Quantum Computing to Quantum Chemistry

## Exercise 1

Consider the following Hamiltonian $$K = \frac{1}{3} \sum_{i<j}X_i X_j - \sum_{i=0}^{n-1}Z_i,$$ where n is the number of qubits (or wires, in Pennylane language), $X_i$ and $Z_i$ are the
Pauli-X and Pauli-Z operators acting on the i-th qubit, respectively, and $\sum_{i<j}$ denotes a sum over all pairs (i, j) where i < j. For example, for n = 3, the pairs that contribute to
the sum over i < j are (0, 1) , (0, 2) and (1, 2). Note that indexing starts from 0.

You must implement in Pennylane a function that simulates a quantum circuit where the qubits start in the state |0⟩ and a Hadamard gate H is applied to all of them, producing a final state |ψ⟩.

The function must receive the number of qubits and return the expectation value of K for |ψ⟩.

First we create a function to generate the expected hamiltonian K

In [2]:
def hamiltonian(n_qubits):
    # We create a list of coefficients. As defined by the hamiltonian K
    # we need two types of coefficients, 1/3 for the operatos XX and -1 for the operator Z.
    # We then define two different lists and concatenate them using the '+' operator from Python.
    # there is a total of (n_qubits-1) * (n_qubits/2) coefficients with value 1/3 and
    # n_qubits operators with value -1.
    coeffs = ([1/3]*((n_qubits-1)*n_qubits//2)) + ([-1]*n_qubits)
    
    # Pennylane allows to describe the Hamiltonian as a sum of operators. We generate the observable XX using
    # the operator '@'. The observable Z can be obtained directly from Pennylane.
    obs = [qml.X(i) @ qml.X(j) for i in range(n_qubits) for j in range(i+1, n_qubits)] + [qml.Z(i) for i in range(n_qubits)]
    
    # We finally generate the Hamiltonian with the Pennylane built-in function.
    return qml.Hamiltonian(coeffs, obs)

We create a function that generates the necessary quantum circuit that returns the expectation value of K.

In [3]:
def quantum_circuit(n_qubits, H):
    # Apply Hadamard in all qubits
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)
    
    # Return the expectation value of Hamiltonian H using the Pennylane built-in function.
    return qml.expval(H)

In [4]:
n = 8 # number of qubits
K = hamiltonian(n) # Hamiltonian K
dev = qml.device('default.qubit', wires=n) # Pennylane device
expectation_value = qml.QNode(quantum_circuit, dev) # Explicitly declaring the Pennylane QNode

print(qml.draw(expectation_value)(n, K))

0: ──H─┤ ╭<𝓗>
1: ──H─┤ ├<𝓗>
2: ──H─┤ ├<𝓗>
3: ──H─┤ ├<𝓗>
4: ──H─┤ ├<𝓗>
5: ──H─┤ ├<𝓗>
6: ──H─┤ ├<𝓗>
7: ──H─┤ ╰<𝓗>


In [5]:
expectation_value(n, K)

tensor(9.33333333, requires_grad=True)

## Challenge 2a

The calculation of molecular properties is crucial in scientific and technological fields, such as chemistry and the pharmaceutical industry. In particular, the ground state energy of a molecule provides valuable insights into its properties, including stability and reactivity.

You must implement a function in Pennylane that simulates the Variational Quantum Eigensolver (VQE). This function should take as input a six-entry array of floating-point numbers, representing the x, y and z coordinates of two hydrogen atoms. The first three entries correspond to the coordinates of the first hydrogen atom, and the remaining three entries represent the coordinates of the second hydrogen atom. The function should return the ground state energy of the molecule. *Also explain why the Pauli-Z is necessary in Jordan-Wigner transformation*.

**Observation: Explicitly define the STO-3G basis function set, multiplicity=1, charge=0 and Jordan-Wigner mapping in the code.**

### Jordan-Wigner Transformation

The **Jordan-Wigner transformation** is used to map fermionic operators onto qubit operators. This is necessary because quantum computers work with qubits rather than fermions. The **Pauli-Z operator** plays a crucial role in this mapping due to the need to preserve the anti-commutation relations of fermionic operators.

For Fermionic operators on the same site we have the following anti-commutation relation:
$$\{f_j, f_j^\dagger\} = 1,$$
and for Fermionic operators on the different sites we have the following anti-commutation relations:
$$\quad \{f_j, f_k\} = \{f_j^\dagger, f_k^\dagger\} = \{f_j, f_k^\dagger\} = 0$$
where $f_j^\dagger$ and $f_j$ are fermionic creation and annihilation operators. To represent these operators on a qubit, we need to ensure that their anti-commutation properties are preserved. The Jordan-Wigner transformation achieves this by converting fermionic creation and annihilation operators into qubit operators using Pauli matrices.

For a single fermionic mode $j$, the Jordan-Wigner transformation maps the creation ($f_j^\dagger$) and annihilation ($f_j$) operators as follows:
$$
f_j^\dagger = \frac{1}{2} (X_j - iY_j) \otimes Z_0 \otimes Z_1 \otimes \dots \otimes Z_{j-1}
$$
$$
f_j = \frac{1}{2} (X_j + iY_j) \otimes Z_0 \otimes Z_1 \otimes \dots \otimes Z_{j-1}
$$
where $X_j$, $Y_j$, and $Z_j$ are the Pauli operators acting on the $j$-th qubit, and the $Z_0 \otimes Z_1 \otimes \dots \otimes Z_{j-1}$ string of Pauli-Z operators appears for all qubits before the $j$-th one.

### Why the Pauli-Z is Necessary:
The Pauli-Z operators are essential in the Jordan-Wigner transformation because they enforce the correct anti-commutation relations between fermionic creation and annihilation operators when mapped onto qubits. This anti-commutation property is not naturally present in qubit operators like $X$, $Y$, and $Z$, which follow commutation relations. To ensure that qubit operators reflect the anti-commutation of fermionic operators, a string of Pauli-Z operators is introduced in the Jordan-Wigner transformation. These Pauli-Z operators essentially track whether a fermion has been "created" or "annihilated" on other qubits, ensuring that the proper sign change occurs. Without these Pauli-Z strings, the qubit representation would not correctly reflect the physics of fermions.

We define a function that will create the ansatz of VQE.

In [6]:
def cost(weights, n_qubits, hf_state, singles, doubles, hamiltonian):
    # Generate the necessary operations with the Pennylane built-in function.
    qml.AllSinglesDoubles(weights, range(n_qubits), hf_state, singles, doubles)
    
    # Return the expectation value of Hamiltonian H using the Pennylane built-in function.
    return qml.expval(hamiltonian)

In [9]:
def H2_ground(coordinates):
    symbols = ['H', 'H'] # Symbols for the H2 Molecule
    coordinates = np.array(coordinates) # Cast coordinates do Numpy Array
    molecule = qml.qchem.Molecule(symbols, coordinates, charge=0, mult=1, basis_name="STO-3G") # Generate the H2 molecule
    H, qubits = qml.qchem.molecular_hamiltonian(molecule, mapping='jordan_wigner') # Create the Hamiltonian from H2
    electrons = 2
    hf = qml.qchem.hf_state(electrons, qubits) # Generate the Hartree-Fock State
    n_qubits = len(H.wires)

    # singles and doubles are used to make the AllSinglesDoubles template
    singles, doubles = qml.qchem.excitations(electrons, n_qubits)

    dev = qml.device("default.qubit", wires=n_qubits) # Define the Pennylane device
    qc = qml.QNode(cost, dev) # Explicitly declare the QNode
    
    opt = qml.AdamOptimizer(0.5)
    
    #Choose an initial parameter for the variation circuit (angle of Givens rotation)
    weights = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)
    
    delta_E = 10
    tol = 1e-5
    it = 0
    #Repetition of optimization until convergence
    while abs(delta_E) > tol:
        it += 1
        weights, prev_energy = opt.step_and_cost(qc, weights, n_qubits=n_qubits, hf_state=hf, singles=singles, doubles=doubles, hamiltonian=H)
        new_energy = qc(weights, n_qubits=n_qubits, hf_state=hf, singles=singles, doubles=doubles, hamiltonian=H)
        delta_E = new_energy - prev_energy
        if it % 5 == 0:
            print(f"Step = {it},  Energy = {new_energy:.6f} Ha")
    print(f"We took {it} iterations until convergence.")

    return new_energy

In [10]:
H2_ground(coordinates = [0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

Step = 5,  Energy = -0.999699 Ha
Step = 10,  Energy = -1.086777 Ha
Step = 15,  Energy = -1.099701 Ha
Step = 20,  Energy = -1.119729 Ha
Step = 25,  Energy = -1.136143 Ha
Step = 30,  Energy = -1.131232 Ha
Step = 35,  Energy = -1.132164 Ha
Step = 40,  Energy = -1.134713 Ha
Step = 45,  Energy = -1.134653 Ha
Step = 50,  Energy = -1.134836 Ha
Step = 55,  Energy = -1.135545 Ha
Step = 60,  Energy = -1.135995 Ha
Step = 65,  Energy = -1.136111 Ha
We took 66 iterations until convergence.


tensor(-1.1361173, requires_grad=True)

## Challenge 2b

For a closed spin chain with a transverse magnetic field of intensity h, the Hamiltonian is: $$H = -\sum_{i=1}^N Z_i \otimes z_{i+1} -h \sum_{i=1}^N X_i,$$ where $Z_i$ and $X_i$ are the Pauli-Z and Pauli-X operators acting on the i-th spin site, respectively. In a closed chain, the site N + 1 is identified with the first site.


You must implement a variational quantum algorithm in Pennylane that, for a given value of the transverse magnetic field h, computes the ground state energy of the spin chain with N = 4 sites. The magnetic field intensity h should be passed as an input to the algorithm.

We define a function to generate the Hamiltonian of a closed chain

In [11]:
def transverse_ising_closed(h, n_qubits):
    coeffs = ([-1]*n_qubits) + ([-h]*n_qubits)
    obs = [qml.Z(i) @ qml.Z(i+1) for i in range(n_qubits-1)] + [qml.Z(n_qubits-1) @ qml.Z(0)] + [qml.X(i) for i in range(n_qubits)]
    
    return qml.Hamiltonian(coeffs, obs)

We define a function for the ansatz to be used in VQE.

In [12]:
def ansatz(params, H):
    params = params.reshape(-1, len(H.wires))
    
    for w in H.wires:
        qml.Hadamard(wires=w)
    
    for i, p in enumerate(params[:len(params)//2]):
        if i%2 ==0:
            qml.AngleEmbedding(features=p, wires=H.wires, rotation='Y')
        else:
            qml.AngleEmbedding(features=p, wires=H.wires, rotation='Z')

    for w in H.wires[:-1]:
        qml.CNOT(wires=[w,w+1])
    
    for i, p in enumerate(params[len(params)//2:]):
        if i%2 ==0:
            qml.AngleEmbedding(features=p, wires=H.wires, rotation='Y')
        else:
            qml.AngleEmbedding(features=p, wires=H.wires, rotation='Z')
    
    return qml.expval(H)


In [18]:
def ground_energy_ising(h):
    N = 4
    H = transverse_ising_closed(h, N)
    theta = np.random.rand(16)
    opt = qml.AdamOptimizer(0.5)

    dev = qml.device("default.qubit", wires=N)
    qc = qml.QNode(ansatz, dev)
    
    delta_E = 10
    results = []
    tol = 1e-6
    #Repetition of optimization until convergence
    while abs(delta_E) > tol:
        theta, prev_energy = opt.step_and_cost(qc, theta, H=H)
        new_energy = qc(theta, H=H)
        delta_E = new_energy - prev_energy
        results.append(new_energy)
        if len(results) % 5 == 0:
            print(f"Step = {len(results)},  Energy = {new_energy:.6f} Ha")
    
    return results[-1]


In [20]:
ground_energy_ising(h=2.3)

Step = 5,  Energy = -7.663451 Ha
Step = 10,  Energy = -8.935372 Ha
Step = 15,  Energy = -8.956452 Ha
Step = 20,  Energy = -9.414306 Ha
Step = 25,  Energy = -9.298180 Ha
Step = 30,  Energy = -9.414176 Ha
Step = 35,  Energy = -9.475196 Ha
Step = 40,  Energy = -9.480111 Ha
Step = 45,  Energy = -9.495372 Ha
Step = 50,  Energy = -9.506477 Ha
Step = 55,  Energy = -9.517018 Ha
Step = 60,  Energy = -9.520710 Ha
Step = 65,  Energy = -9.525098 Ha
Step = 70,  Energy = -9.529672 Ha
Step = 75,  Energy = -9.530652 Ha
Step = 80,  Energy = -9.531965 Ha
Step = 85,  Energy = -9.532994 Ha
Step = 90,  Energy = -9.533384 Ha
Step = 95,  Energy = -9.533790 Ha
Step = 100,  Energy = -9.534038 Ha


tensor(-9.53403805, requires_grad=True)

## Challenge 2c

The total energy of a quantum system is represented by operator H. We know that an initial quantum state |ψ⟩ evolves over time through the action of the evolution operator U , defined as $$U(t) = exp[-iHt]$$ However, it’s possible to build quantum circuits that approximates the realization of U(t), being one of these methods knows as Trotterization. If we can write H as $$H = \sum_{i=1}^k H_i$$ of a number k of Hermitian operators $H_i$ that do not necessarily commute, we can approximate U(t) via $$U(t) \approx \prod_{j=1}^n\prod_{i=1}^k exp\left( -\frac{iH_it}{n} \right)$$ with each term in the j-product called trotter step. The larger n is, i.e. more trotter
steps, the better the approximation of U that we get.

You must implement in Pennylane a function which applies the evolution operator related to the Hamiltonian $$H = \alpha X\otimes X + \beta Z\otimes Z$$ in the state |00⟩. The function must receive as parameters α, β, the time t and the number of trotter steps n in this order. It must return the probability of measure |00⟩, |01⟩, |10⟩, |11⟩.

**Observation: You may not use PennyLane’s built-in time evolution (qml.ApproxTimeEvolution, qml.evolve) nor qml.QubitUnitary. All the measures must be done in computational base.**

We trotterize the given Hamiltonian H

In [21]:
def trotterize(alpha, beta, time, depth):
    for d in range(depth):
        qml.Hadamard(wires=[0])
        qml.Hadamard(wires=[1])
        qml.CNOT(wires=[0,1])
        qml.RZ(2*alpha*(time/depth), wires=1)
        qml.CNOT(wires=[0,1])
        qml.Hadamard(wires=[0])
        qml.Hadamard(wires=[1])
        
        qml.CNOT(wires=[0,1])
        qml.RZ(2*beta*(time/depth), wires=1)
        qml.CNOT(wires=[0,1])
    
    # Return the probabilities
    return qml.probs(wires=[0,1])

In [23]:
dev = qml.device("default.qubit", wires=2) # Define the Pennylane device
qc = qml.QNode(trotterize, dev) # Explicitly declare the QNode

qc(alpha=0.5, beta=0.8, time=0.2, depth=1).numpy()

array([0.99003329, 0.        , 0.        , 0.00996671])